# 微调 BERT 用于序列级和词元级应用
:label:`sec_finetuning-bert`


在本章的前面部分，
我们为自然语言处理应用设计了不同的模型，
例如基于 RNN、CNN、注意力机制和 MLP 的模型。
当存在空间或时间限制时，
这些模型是有帮助的，
然而，
为每个自然语言处理任务设计特定模型实际上是不可行的。
在 :numref:`sec_bert` 中，
我们介绍了一个预训练模型 BERT，
该模型对于广泛的自然语言处理任务只需进行最小的架构更改。
一方面，
在其提出时，
BERT 在各种自然语言处理任务上改进了当时的最先进水平。
另一方面，
如 :numref:`sec_bert-pretraining` 所述，
原始 BERT 模型的两个版本分别带有 1.1 亿和 3.4 亿个参数。
因此，当有足够的计算资源时，
我们可以考虑
对 BERT 进行微调以应用于下游自然语言处理任务。

接下来，
我们将一部分自然语言处理应用
概括为序列级和词元级。
在序列级别，
我们介绍了如何将文本输入的 BERT 表示转换为输出标签
在单文本分类
以及文本对分类或回归中。
在词元级别，我们将简要介绍新的应用
例如文本标注和问答
并阐明 BERT 如何表示它们的输入并转换为输出标签。
在微调过程中，
不同应用所需的“最小架构更改”是额外的全连接层。
在监督学习下游应用时，
额外层的参数是从头开始学习的，
而预训练 BERT 模型中的所有参数都会被微调。


## 单文本分类

*单文本分类* 将单个文本序列作为输入并输出其分类结果。
除了我们在本章中研究的情感分析之外，
语言可接受性语料库（CoLA）也是一个单文本分类的数据集，
判断给定句子是否语法上可接受 :cite:`Warstadt.Singh.Bowman.2019`。
例如，“I should study.” 是可接受的，但 “I should studying.” 不是。

![微调 BERT 用于单文本分类应用，例如情感分析和测试语言可接受性。假设输入的单文本有六个词元。](../img/bert-one-seq.svg)
:label:`fig_bert-one-seq`

:numref:`sec_bert` 描述了 BERT 的输入表示。
BERT 输入序列明确地表示了单文本和文本对，
其中特殊的分类标记
“&lt;cls&gt;” 用于序列分类，
特殊的分类标记
“&lt;sep&gt;” 标记单文本的结束或分隔一对文本。
如 :numref:`fig_bert-one-seq` 所示，
在单文本分类应用中，
特殊分类标记
“&lt;cls&gt;” 的 BERT 表示编码了整个输入文本序列的信息。
作为输入单文本的表示，
它将被送入一个由全连接（密集）层组成的小型 MLP，
以输出所有离散标签值的分布。


## 文本对分类或回归

我们还在本章中探讨了自然语言推理。
它属于 *文本对分类*，
一种对一对文本进行分类的应用类型。

输入一对文本但输出连续值，
*语义文本相似度* 是一种流行的 *文本对回归* 任务。
该任务测量句子的语义相似性。
例如，在语义文本相似度基准数据集中，
一对句子的相似度得分
是一个从 0（无意义重叠）到 5（意义等效）的有序尺度 :cite:`Cer.Diab.Agirre.ea.2017`。
目标是预测这些分数。
来自语义文本相似度基准数据集的例子包括（句子 1，句子 2，相似度得分）：

* "A plane is taking off.", "An air plane is taking off.", 5.000;
* "A woman is eating something.", "A woman is eating meat.", 3.000;
* "A woman is dancing.", "A man is talking.", 0.000。


![微调 BERT 用于文本对分类或回归应用，例如自然语言推理和语义文本相似度。假设输入的文本对有两个和三个词元。](../img/bert-two-seqs.svg)
:label:`fig_bert-two-seqs`

与 :numref:`fig_bert-one-seq` 中的单文本分类相比，
微调 BERT 用于 :numref:`fig_bert-two-seqs` 中的文本对分类
在输入表示上有所不同。
对于诸如语义文本相似度之类的文本对回归任务，
可以进行一些简单的更改，例如输出连续标签值
并使用均方损失：这些在回归中很常见。


## 文本标注

现在让我们考虑词元级任务，例如 *文本标注*，
其中每个词元都被分配一个标签。
在文本标注任务中，
*词性标注* 根据单词在句子中的角色为其分配词性标签（例如，形容词和限定词）。
例如，
根据宾州树库 II 标签集，
句子 "John Smith 's car is new"
应被标注为
"NNP (名词，专有名词单数) NNP POS (属格结尾) NN (名词，单数或质量名词) VB (动词，原形) JJ (形容词)"。

![微调 BERT 用于文本标注应用，例如词性标注。假设输入的单文本有六个词元。](../img/bert-tagging.svg)
:label:`fig_bert-tagging`

微调 BERT 用于文本标注应用
如 :numref:`fig_bert-tagging` 所示。
与 :numref:`fig_bert-one-seq` 相比，
唯一的区别在于
在文本标注中，输入文本的每个词元的 BERT 表示
都送入相同的额外全连接层以输出词元的标签，
例如词性标签。



## 问答

作为另一个词元级应用，
*问答* 反映了阅读理解的能力。
例如，
斯坦福问答数据集（SQuAD v1.1）
由阅读段落和问题组成，
其中每个问题的答案
只是段落中的一段文本（文本片段），问题正是关于这个段落的 :cite:`Rajpurkar.Zhang.Lopyrev.ea.2016`。
为了说明，
考虑一段话
“Some experts report that a mask's efficacy is inconclusive. However, mask makers insist that their products, such as N95 respirator masks, can guard against the virus.”
和一个问题 “Who say that N95 respirator masks can guard against the virus?”。
答案应该是段落中的文本片段 "mask makers"。
因此，SQuAD v1.1 的目标是预测给定问题和段落对的情况下段落中文本片段的起始和结束位置。

![微调 BERT 用于问答。假设输入的文本对有两个和三个词元。](../img/bert-qa.svg)
:label:`fig_bert-qa`

为了微调 BERT 用于问答，
问题和段落分别作为
BERT 输入的第一和第二个文本序列。
为了预测文本片段的起始位置，
同一个额外的全连接层将
段落中位置 $i$ 的任何词元的 BERT 表示
转换为标量得分 $s_i$。
所有段落词元的这种得分
进一步通过 softmax 操作
转换为概率分布，
使得段落中的每个词元位置 $i$
被赋予成为文本片段起始位置的概率 $p_i$。
预测文本片段的结束位置
与上述相同，只是
其额外全连接层的参数
独立于预测起始位置的参数。
在预测结束位置时，
任何位置 $i$ 的段落词元
都通过同一个全连接层
转换为标量得分 $e_i$。
:numref:`fig_bert-qa`
描绘了微调 BERT 用于问答的过程。

对于问答，
监督学习的训练目标很简单，
即最大化真实起始和结束位置的对数似然。
在预测跨度时，
我们可以计算有效跨度从位置 $i$ 到位置 $j$ ($i \leq j$) 的得分 $s_i + e_j$，
并输出得分最高的跨度。


## 总结

* BERT 需要最小的架构更改（额外的全连接层）来应对序列级和词元级的自然语言处理应用，例如单文本分类（例如，情感分析和测试语言可接受性）、文本对分类或回归（例如，自然语言推理和语义文本相似度）、文本标注（例如，词性标注）和问答。
* 在监督学习下游应用时，额外层的参数是从头开始学习的，而预训练 BERT 模型中的所有参数都会被微调。


## 练习

1. 让我们设计一个新闻文章的搜索引擎算法。当系统收到查询（例如，“冠状病毒爆发期间的石油行业”）时，它应该返回与查询最相关的新闻文章的排名列表。假设我们有一个庞大的新闻文章池和大量的查询。为了简化问题，假设每个查询的最相关文章已经被标记。我们如何在算法设计中应用负采样（见 :numref:`subsec_negative-sampling`）和 BERT？
1. 我们如何利用 BERT 来训练语言模型？
1. 我们能否利用 BERT 进行机器翻译？

[讨论](https://discuss.d2l.ai/t/396)